In [1]:
a = 3

3

In [6]:
?randn

search: randn rand transcode macroexpand @macroexpand1 @macroexpand



```
randn([rng=GLOBAL_RNG], [T=Float64], [dims...])
```

Generate a normally-distributed random number of type `T` with mean 0 and standard deviation 1. Optionally generate an array of normally-distributed random numbers. The `Base` module currently provides an implementation for the types [`Float16`](@ref), [`Float32`](@ref), and [`Float64`](@ref) (the default), and their [`Complex`](@ref) counterparts. When the type argument is complex, the values are drawn from the circularly symmetric complex normal distribution of variance 1 (corresponding to real and imaginary part having independent normal distribution with mean zero and variance `1/2`).

# Examples

```jldoctest
julia> using Random

julia> rng = MersenneTwister(1234);

julia> randn(rng, ComplexF64)
0.6133070881429037 - 0.6376291670853887im

julia> randn(rng, ComplexF32, (2, 3))
2×3 Array{Complex{Float32},2}:
 -0.349649-0.638457im  0.376756-0.192146im  -0.396334-0.0136413im
  0.611224+1.56403im   0.355204-0.365563im  0.0905552+1.31012im
```


In [9]:
using Distributions

┌ Info: Precompiling Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1260


ZeroMeanFullNormal(
dim: 2
μ: [0.0, 0.0]
Σ: [9.0 0.0; 0.0 0.1]
)


simulate_AR1 (generic function with 1 method)

In [36]:
R = [0.9 0.001; 0.001 0.9]
Σ = [0.1 0.001; 0.001 0.1]

2×2 Array{Float64,2}:
 0.1    0.001
 0.001  0.1

In [61]:
function simulate_AR1(R::Matrix{Float64}, Σ::Matrix{Float64}; s0::Vector{Float64}=zeros(size(Σ,1)), T=100)
    dis = MvNormal(Σ)
    n = size(R,1)
    println(s0)
    sim = zeros(n,T)
    sim[:,1] = s0
    for t=1:T-1
        s = sim[:,t]
        ϵ = rand(dis)
        sim[:,t+1] = R*s + ϵ
    end
    return sim
    
end
@time sim = simulate_AR1(R, Σ;T=100000)

[0.0, 0.0]
  0.029773 seconds (500.12 k allocations: 39.676 MiB, 14.35% gc time)


2×100000 Array{Float64,2}:
 0.0  -0.112207    0.154457  0.643342  …  -0.456298  -0.78983  -0.428279
 0.0  -0.0110298  -0.122583  0.558112     -0.524118  -0.89321  -0.41718

[0.0, 0.0]
  0.029974 seconds (500.12 k allocations: 39.676 MiB, 14.78% gc time)


2×100000 Array{Float64,2}:
 0.0   0.234563   0.112983  -0.0440957  …  -0.375937   -0.506763  -0.413832
 0.0  -0.601531  -1.36904   -1.77055        0.0687324   0.096512  -0.183513

In [50]:
size(sim)

(2, 100000)

In [62]:
function simulate_AR1_2(R::Matrix{Float64}, Σ::Matrix{Float64}; s0::Vector{Float64}=zeros(size(Σ,1)), T=100)
    dis = MvNormal(Σ)
    n = size(R,1)
    println(s0)
    sim = zeros(T,n)
    sim[1,:] = s0
    for t=1:T-1
        s = sim[t,:]
        ϵ = rand(dis)
        sim[t+1,:] = R*s + ϵ
    end
    return sim
    
end
@time sim = simulate_AR1_2(R, Σ;T=100000);

[0.0, 0.0]
  0.030326 seconds (500.12 k allocations: 39.676 MiB, 14.96% gc time)


In [52]:
using StaticArrays

In [75]:
function simulate_AR1_2(R::SMatrix, Σ::SMatrix; s0::SVector=zero(SVector(Σ[:,1]...)), T=100)
    dis = MvNormal(Σ)
    res = zeros(typeof(s0), T)
    res[1] = s0
    for t=1:T-1
        s = sim[t]
        ϵ = rand(dis)
        sim[t+1] = R*s + ϵ
    end
end
@SMatrix R
@time sim = simulate_AR1_2(R, Σ;T=100000)

ErrorException: The size of type `SArray{Tuple{S1,S2},T,2,L} where L where T where S2 where S1` is not known.

If you were trying to construct (or `convert` to) a `StaticArray` you
may need to add the size explicitly as a type parameter so its size is
inferrable to the Julia compiler (or performance would be terrible). For
example, you might try

    m = zeros(3,3)
    SMatrix(m)            # this error
    SMatrix{3,3}(m)       # correct - size is inferrable
    SArray{Tuple{3,3}}(m) # correct, note Tuple{3,3}


In [76]:
SMatrix(R)

ErrorException: The size of type `SArray{Tuple{S1,S2},T,2,L} where L where T where S2 where S1` is not known.

If you were trying to construct (or `convert` to) a `StaticArray` you
may need to add the size explicitly as a type parameter so its size is
inferrable to the Julia compiler (or performance would be terrible). For
example, you might try

    m = zeros(3,3)
    SMatrix(m)            # this error
    SMatrix{3,3}(m)       # correct - size is inferrable
    SArray{Tuple{3,3}}(m) # correct, note Tuple{3,3}


In [72]:
zero(SVector(Σ[:,1]...))

2-element SArray{Tuple{2},Float64,1,2} with indices SOneTo(2):
 0.0
 0.0